# Probabilistic Imaginary Time-Evolution

We are going to implement the algorithm outlined in [Optimal Scheduling in probabilistic imaginary-time evolution on a quantum computer](https://arxiv.org/abs/2305.04600). It approximates the imaginary time-evolution of a system using controlled time-evolutions followed by a measurement.

The circuit for the algorithm is

![PITE Circuit](PITE_Circuit.png "Phase Estimation Circuit")


Repeatedly applying the above circuit will drive the system to it's ground state at the cost of exponentially decreasing probability.

In [72]:
from openfermion import get_sparse_operator
import numpy as np

from tangelo.linq import Circuit, Gate, get_backend
from tangelo.toolboxes.ansatz_generator.ansatz_utils import trotterize
from tangelo.molecule_library import mol_H2_sto3g
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit

fe_op = mol_H2_sto3g.fermionic_hamiltonian
qu_op = fermion_to_qubit_mapping(fe_op, "scbk", n_spinorbitals=4, n_electrons=2)

ham = get_sparse_operator(qu_op).toarray()

hf_circ = get_reference_circuit(4, 2, "scbk", True, 0)

eigs, vecs = np.linalg.eigh(ham)
sim = get_backend("cirq")
_, hf_state = sim.simulate(hf_circ, return_statevector=True)

In [228]:
w_circ = Circuit([Gate("RZ", 2, parameter = np.pi/2), Gate("RY", 2, parameter=np.pi/2), Gate("PHASE", 2, parameter = np.pi/4)])

def pite_circuit(qubit_op, tau, theta):
    Urte = trotterize(qubit_op, tau, control=2, trotter_order=4, n_trotter_steps=4)
    circuit = (Circuit([Gate("H", 2)])+w_circ+
               Circuit([Gate("X", 2)]) + Urte + Circuit([Gate("X", 2)])+
               Urte.inverse()+
               Circuit([Gate("RZ", 2, parameter=-2*theta)])+
               w_circ.inverse()+
               Circuit([Gate("MEASURE", 2)]))
    return circuit


In [229]:
n_steps = 4
dmeas = "0"*n_steps
m0 = 1/np.sqrt(2)+0.00001
kappa = np.sign(m0-1/np.sqrt(2))


theta0 = kappa*np.arccos((m0+np.sqrt(1-m0**2))/np.sqrt(2))
s1 = m0/np.sqrt(1-m0**2)

print(theta0, s1)


pite = hf_circ+pite_circuit(qubit_op=qu_op, tau=s1, theta=-theta0)*n_steps
print(pite.counts_n_qubit)
f, sv = sim.simulate(pite, desired_meas_result=dmeas, return_statevector=True)
print(pite.success_probabilities)
print(sv)
sv = np.reshape(sv, (4, 2))[:, 0]


1.4142246114952058e-05 1.000028284871264
{1: 1646, 2: 2560}
{'0000': 0.5953000745830743}
[-1.11636966e-01+1.13835121e-15j  0.00000000e+00+0.00000000e+00j
 -3.20132297e-16-7.93279724e-17j  0.00000000e+00+0.00000000e+00j
 -1.68242030e-16-2.73487488e-16j  0.00000000e+00+0.00000000e+00j
  9.93749057e-01-7.74670176e-15j  0.00000000e+00+0.00000000e+00j]


In [235]:
print(f'initial overlap = {abs(np.dot(hf_state, vecs[:, 0]))}')
print(f'final overlap = {abs(np.dot(sv.conj(), vecs[:, 0]))}')


initial overlap = 0.9936146058054715
final overlap = 0.9999992418252036
[-0.11163679+2.25169845e-16j  0.        +0.00000000e+00j
  0.        +0.00000000e+00j  0.99374904-1.65528878e-15j]
[ 0.11282737+0.j  0.        +0.j  0.        +0.j -0.99361461+0.j]


We can also use Quantum Signal Processing for the time-evolution. Although, these times are so short that it does not make sense

In [254]:
from tangelo.toolboxes.circuits.qsp import get_qsp_hamiltonian_simulation_circuit, get_qsp_hamiltonian_simulation_qubit_list
qsp_qubits = get_qsp_hamiltonian_simulation_qubit_list(qu_op)
cq = qsp_qubits[-1] + 1
w_circ = Circuit([Gate("RZ", cq, parameter = np.pi/2), Gate("RY", cq, parameter=np.pi/2), Gate("PHASE", cq, parameter = np.pi/4)])

def pite_circuit(qubit_op, tau, theta):
    Urte = get_qsp_hamiltonian_simulation_circuit(qubit_op, tau, eps=0.001, control=cq)
    circuit = (Circuit([Gate("H", cq)])+w_circ+
               Circuit([Gate("X", cq)]) + Urte + Circuit([Gate("X", cq)])+
               Urte.inverse()+
               Circuit([Gate("RZ", cq, parameter=-2*theta)])+
               w_circ.inverse()+
               Circuit([Gate("MEASURE", cq)]))
    return circuit


In [255]:
n_steps = 4
dmeas = "0"*n_steps
m0 = 1/np.sqrt(2)-0.00001
kappa = np.sign(m0-1/np.sqrt(2))


theta0 = kappa*np.arccos((m0+np.sqrt(1-m0**2))/np.sqrt(2))
s1 = m0/np.sqrt(1-m0**2)

print(theta0, s1)


pite = hf_circ+pite_circuit(qubit_op=qu_op, tau=s1, theta=-theta0)*n_steps
print(pite.counts_n_qubit)
f, sv = sim.simulate(pite, desired_meas_result=dmeas, return_statevector=True)
print(pite.success_probabilities)
sv = np.reshape(sv, (4, 2**7))[:, 0]

-1.4142042003054168e-05 0.9999717163287355
5.745046242204417
R=2
[PolyCosineTX] rescaling by 0.5.
5.745046242204417
R=2
[PolySineTX] rescaling by 0.5.
{1: 25806, 2: 6280, 4: 2632, 7: 7560, 6: 48}
{'0000': 0.595559054541099}


In [256]:
print(f'initial overlap = {abs(np.dot(hf_state, vecs[:, 0]))}')
print(f'final overlap = {abs(np.dot(sv.conj(), vecs[:, 0]))}')

initial overlap = 0.9936146058054715
final overlap = 0.999999240878011
